In [1]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

# setup django
import os
import sys
sys.path.append('../../')
os.environ["DJANGO_SETTINGS_MODULE"] = "settings"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "settings")
import django
django.setup()

from apps.document.models import *
from apps.rawdb.signals import reindex_on_doc_type_change
import pandas as pd

### List Existing Document Types

In [2]:
def info():
    data = []
    for i in DocumentType.objects.order_by('code'):
        data.append((i.pk, i.fields.count(), i.search_fields.count(),
                     DocumentFieldDetector.objects.filter(field__document_type=i).count(), i.code, i.title))
    return pd.DataFrame.from_records(data, columns=['pk', 'fields', 'search_fields', 'detectors', 'code', 'title'])
info()

,pk,fields,search_fields,detectors,code,title
0,232b4976-e406-4769-913d-e8d8a30cb2cc,37,7,34,atticus,Atticus
1,302ad283-ba63-4a37-903b-e9e9d839942f,1,0,1,audit_provision,Audit Provision
2,a7d60e7d-d23f-4ae7-9186-7e7bd96b8b94,1,0,0,bond,bond
3,b72f0fed-2381-4ca6-bb6b-d436460ca9da,6,6,6,contract.custody,Contract Custody
4,af0d1731-3d09-4cc7-ae9a-4952a77ab43e,5,5,5,contract.SellersWarrantiesAndServ,Seller's Warranties and Servicing Agreement
5,68f992f1-dba3-4dc0-a815-4d868b23c5b4,0,0,0,document.GenericDocument,Generic Document
6,fd869060-92cb-422c-991a-985ede2d50b7,9,8,10,employment.EmploymentAgrement,Employment Agreement
7,78425541-fe21-4ede-9445-ca88ceb3fc4c,20,7,26,lease.LeaseDocument,Lease Document
8,0e5c7194-4484-4720-9fd1-2df15b109e36,1,1,1,libor.CreditAgreement,Non-Syndicated Credit Agreement
9,316f744b-bca0-4b86-af35-15689a0ec99b,1,1,0,libor.CreditAgreement.copy,Non-Syndicated Credit Agreement-COPY


### Choose Document Type PK to clone, Provide new code and title

In [9]:
source_document_type_id = '81ef20fd-4b92-4f53-a386-7516504ac329'
new_document_type_code = 'libor_v3_syndicated_credit'
new_document_type_title = 'LIBOR V3 Syndicated Credit'

### Clone

In [10]:
dt = DocumentType.objects.get(pk=source_document_type_id)
dt_fields = dt.fields.all()
dt_search_field_codes = dt.search_fields.values_list('code', flat=True)

dt.pk = None
dt.code = new_document_type_code
dt.title = new_document_type_title
dt.save()

# copy fields
depends_on_fields_map = {}
for field in dt_fields:
    field_detectors = field.field_detectors.all()
    depends_on_fields_codes = field.depends_on_fields.values_list('code', flat=True)
    field.pk = None
    field.document_type = dt
    field.save()
    depends_on_fields_map[field] = depends_on_fields_codes

    for detector in field_detectors:
        detector.pk = None
        detector.field = field
        detector.save()
                
# copy depends_on_fields
for field, dependent_field_codes in depends_on_fields_map.items():
    dependent_fields = dt.fields.filter(code__in=dependent_field_codes)
    field.depends_on_fields.set(dependent_fields)

# copy search_fields
search_fields = dt.fields.filter(code__in=dt_search_field_codes)
dt.search_fields.set(search_fields)

reindex_on_doc_type_change(dt)

### next you can list Existing Document Types again to ensure new Type created

In [ ]:
info()